In [1]:
# Import standard packages
import os
import multiprocessing

os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

In [3]:
from transformers import AutoTokenizer, AutoModel

In [4]:
with open('huggingface.txt', 'r') as f:
    token = f.read()

In [5]:
model = AutoModel.from_pretrained("google-bert/bert-base-cased")

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

In [6]:
model = AutoModel.from_pretrained("meta-llama/Llama-3.2-1B-Instruct", token = token)

: 

In [4]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B-Instruct", token = token)

In [7]:
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B-Instruct", token = token)

: 

In [5]:
input_ids = tokenizer("Hello.", return_tensors="pt").to("cuda")

In [6]:
input_ids

{'input_ids': tensor([[128000,   9906,     13]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1]], device='cuda:0')}

In [13]:
output = model.generate(**input_ids)

In [14]:
print(tokenizer.decode(output[0], skip_special_tokens=True))

Hello. I'm a 20 year old male. I'm a student at


In [80]:
llm = ChatOllama(
    model="llama3.2:latest",
    temperature=0.7,
    n_gpu_layers=-1,
    n_threads=multiprocessing.cpu_count() - 1,
    verbose=False,
    seed=-1 # Random behavior so will not output the same thing always
)

In [76]:
def traverse_directory(directory):
    for root, _, files in os.walk(directory):
        for f in files:
            yield os.path.join(root, f)
            
file_paths = []
for f in traverse_directory("test_dirs"):
    file_paths.append(f)
    
str(file_paths)

"['test_dirs/images/dog.avif', 'test_dirs/images/cat.jpg', 'test_dirs/src/create_user.py', 'test_dirs/src/dogs.py', 'test_dirs/src/access_db.py']"

In [89]:
query = "'Find me all files related to databases.'"
question = "Is the following file related to the query " + query + "? Respond only with 'yes' or 'no':"

for f in file_paths:
    print(f, llm.invoke(question + " " + f).content)

test_dirs/images/dog.avif No
test_dirs/images/cat.jpg No
test_dirs/src/create_user.py No.
test_dirs/src/dogs.py No.
test_dirs/src/access_db.py Yes.


In [38]:
msg = llm.invoke('From the following list of files, which one(s) are most related to dogs? Return only the file paths and nothing else.' + str(file_paths))
msg.content

"'test_dirs/images/dog.avif', 'test_dirs/src/dogs.py'"

### KILT dataset

In [5]:
ds = load_dataset("facebook/kilt_tasks", "nq")

Generating test split: 100%|██████████| 1444/1444 [00:00<00:00, 350394.85 examples/s]


In [42]:
from kilt.knowledge_source import KnowledgeSource

In [43]:
ks = KnowledgeSource()

ks.get_num_pages()

5903530

In [45]:
# get page by id
page = ks.get_page_by_id(27097632)
page

{'_id': '27097632',
 'wikipedia_id': '27097632',
 'wikipedia_title': 'Anarchy',
 'text': ['Anarchy\n',
  'Anarchy refers to a society, entity, group of people, or a single person that rejects hierarchy. The word originally meant leaderlessness, but Pierre-Joseph Proudhon adopted the term in his 1840 treatise "What Is Property?" to refer to anarchism, a new political philosophy which advocates stateless societies based on voluntary associations.\n',
  'In practical terms, anarchy can refer to the curtailment or abolition of traditional forms of government and institutions. It can also designate a nation—or anywhere on earth that is inhabited—that has no system of government or central rule. Anarchy is primarily advocated by individual anarchists who propose replacing government with voluntary institutions.\n',
  'Section::::Etymology.\n',
  'The word anarchy comes from the ancient Greek ἀναρχία ("anarchia") which combines ἀ ("a"), "without, not" and ἀρχή ("arkhi"), "ruler, authority". T

In [47]:
page['anchors']

[{'text': 'society',
  'href': 'society',
  'paragraph_id': 1,
  'start': 20,
  'end': 27},
 {'text': 'group of people',
  'href': 'group%20of%20people',
  'paragraph_id': 1,
  'start': 37,
  'end': 52},
 {'text': 'hierarchy',
  'href': 'hierarchy',
  'paragraph_id': 1,
  'start': 86,
  'end': 95},
 {'text': 'leader',
  'href': 'leader',
  'paragraph_id': 1,
  'start': 123,
  'end': 129},
 {'text': 'Pierre-Joseph Proudhon',
  'href': 'Pierre-Joseph%20Proudhon',
  'paragraph_id': 1,
  'start': 143,
  'end': 165},
 {'text': 'What Is Property?',
  'href': 'What%20Is%20Property%3F',
  'paragraph_id': 1,
  'start': 205,
  'end': 222},
 {'text': 'anarchism',
  'href': 'anarchism',
  'paragraph_id': 1,
  'start': 236,
  'end': 245},
 {'text': 'political philosophy',
  'href': 'political%20philosophy',
  'paragraph_id': 1,
  'start': 253,
  'end': 273},
 {'text': 'stateless',
  'href': 'State%20%28polity%29',
  'paragraph_id': 1,
  'start': 290,
  'end': 299},
 {'text': 'voluntary association'

In [48]:
page = ks.get_page_by_title("Michael Jordan")

1.3 seconds is not inconsequential. It shows we cannot just search through all titles blindly. We need to do a search based on the text of a query.